校验kmeans的结果，合成信号值， 多头信号1 数据应该集中数据头部，  空头信号-1 数据应该集中在数据尾部  中间处于不动即0

In [1]:
import os
import pandas as pd
from lumina.genetic.fusion import Rotor
from lumina.genetic import Actuator

In [4]:
code = 'im'
def fetch_rotor(name):
    base_path = '/workspace/worker/pj/Chrono/lumina/abily/records/temp/aicso2/ims/kmeans'
    rotor = Rotor.from_pickle(
            path=os.path.join(base_path, code.lower()), name=name)
    return rotor

from kdutils.file import fetch_file_data
base_path = '/workspace/worker/pj/Chrono/lumina/abily/records/'
#factors_data = fetch_file_data(base_path=base_path,
#                          method='aicso2',
#                          g_instruments='ims',
#                          datasets=['train_data', 'val_data'])

def fetch_data():
    base_path = '/workspace/worker/pj/Chrono/lumina/abily/records/'
    filename = os.path.join(base_path, 'aicso2', 'ims', 'merge',"val_data.feather")
    factors_data = pd.read_feather(filename).sort_values(
        by=['trade_time', 'code'])
    return factors_data

def create_postions(k_split, filter_strategies, total_data):
    actuator = Actuator(k_split=k_split)

    strategies_data = actuator.calculate(strategies_infos=filter_strategies,
                                         total_data=total_data)
    return strategies_data


def merge_signals(strategies_data, filter_strategies):
    actuator = Actuator(k_split=4)
    weights_data = actuator.fitness_weight(strategies_infos=filter_strategies)
    positions_data = actuator.fitness_signal(
        strategies_infos=filter_strategies,
        strategies_data=strategies_data,
        weights_data=weights_data)
    return positions_data, weights_data

total_data = fetch_file_data(base_path=base_path,
                          method='aicso2',
                          g_instruments='ims',
                          datasets=['train_data', 'val_data'])
total_data['trade_time'] = pd.to_datetime(total_data['trade_time'])
basic_rotor = fetch_rotor('1012936101')

strategies_data = create_postions(k_split=4,
                                      filter_strategies=basic_rotor.strategies,
                                      total_data=total_data)
positions_data, weights_data = merge_signals(strategies_data,
                                                 basic_rotor.strategies)
positions_data.name = 'value'
positions_data = positions_data.reset_index().set_index('trade_time')
positions_data.head()

predict groups model::8.22%

/workspace/worker/env/lingua/lib/python3.8/site-packages/Finance_Ultron-1.7.8-py3.8-linux-x86_64.egg/ultron/factor/genetic/geneticist/operators.py:320: RuntimeWarning: divide by zero encountered in true_divide
  return eval(expression).transform(total_data,


predict groups model::19.18%

/workspace/worker/env/lingua/lib/python3.8/site-packages/Finance_Ultron-1.7.8-py3.8-linux-x86_64.egg/ultron/factor/genetic/geneticist/operators.py:320: RuntimeWarning: divide by zero encountered in true_divide
  return eval(expression).transform(total_data,
/workspace/worker/env/lingua/lib/python3.8/site-packages/Finance_Ultron-1.7.8-py3.8-linux-x86_64.egg/ultron/factor/genetic/geneticist/operators.py:320: RuntimeWarning: invalid value encountered in true_divide
  return eval(expression).transform(total_data,


predict groups model::42.47%

...

/workspace/worker/env/lingua/lib/python3.8/site-packages/Finance_Ultron-1.7.8-py3.8-linux-x86_64.egg/ultron/factor/genetic/geneticist/operators.py:320: RuntimeWarning: invalid value encountered in true_divide
  return eval(expression).transform(total_data,


predict groups model::91.78%

/workspace/worker/env/lingua/lib/python3.8/site-packages/Finance_Ultron-1.7.8-py3.8-linux-x86_64.egg/ultron/factor/genetic/geneticist/operators.py:320: RuntimeWarning: divide by zero encountered in true_divide
  return eval(expression).transform(total_data,


predict groups model::100.0%

,code,value
trade_time,,
2022-07-25 09:30:00,IM,0.0000
2022-07-25 09:31:00,IM,0.0000
2022-07-25 09:32:00,IM,0.0000
2022-07-25 09:33:00,IM,0.0000
2022-07-25 09:34:00,IM,-0.0051


In [5]:
roto1 = fetch_rotor('1012936101')
signal_data = roto1.predict(positions_data)
signal_data.name = 'signal'
t1_dt = pd.concat([positions_data, signal_data], axis=1)
short = t1_dt[t1_dt['signal']==-1].value.max()
long = t1_dt[t1_dt['signal']==1].value.max()
zero = t1_dt[t1_dt['signal']==0].value.max()
print("short:{0}, long:{1}, zero:{2}, mapping:{3}".format(short, long, zero, roto1._best_mapping))


short:0.04752687979410584, long:0.4159098338216611, zero:-0.1020546979815696, mapping:{0: -1, 1: 0, 2: 1}


In [6]:
roto1 = fetch_rotor('1019752472')
signal_data = roto1.predict(positions_data)
signal_data.name = 'signal'
t1_dt = pd.concat([positions_data, signal_data], axis=1)
short = t1_dt[t1_dt['signal']==-1].value.max()
long = t1_dt[t1_dt['signal']==1].value.max()
zero = t1_dt[t1_dt['signal']==0].value.max()
print("short:{0}, long:{1}, zero:{2}, mapping:{3}".format(short, long, zero, roto1._best_mapping))


short:-0.1020546979815696, long:0.04752687979410584, zero:0.4159098338216611, mapping:{0: 1, 1: -1, 2: 0}


In [7]:
roto1 = fetch_rotor('1022579706')
signal_data = roto1.predict(positions_data)
signal_data.name = 'signal'
t1_dt = pd.concat([positions_data, signal_data], axis=1)
short = t1_dt[t1_dt['signal']==-1].value.max()
long = t1_dt[t1_dt['signal']==1].value.max()
zero = t1_dt[t1_dt['signal']==0].value.max()
print("short:{0}, long:{1}, zero:{2}, mapping:{3}".format(short, long, zero, roto1._best_mapping))


short:0.4159098338216611, long:-0.1020546979815696, zero:0.04752687979410584, mapping:{0: 0, 1: 1, 2: -1}


In [8]:
roto1 = fetch_rotor('1056294026')
signal_data = roto1.predict(positions_data)
signal_data.name = 'signal'
t1_dt = pd.concat([positions_data, signal_data], axis=1)
short = t1_dt[t1_dt['signal']==-1].value.max()
long = t1_dt[t1_dt['signal']==1].value.max()
zero = t1_dt[t1_dt['signal']==0].value.max()
print("short:{0}, long:{1}, zero:{2}, mapping:{3}".format(short, long, zero, roto1._best_mapping))


short:0.4159098338216611, long:0.04752687979410584, zero:-0.1020546979815696, mapping:{0: 1, 1: 0, 2: -1}


In [9]:
roto1 = fetch_rotor('1073079186')
signal_data = roto1.predict(positions_data)
signal_data.name = 'signal'
t1_dt = pd.concat([positions_data, signal_data], axis=1)
short = t1_dt[t1_dt['signal']==-1].value.max()
long = t1_dt[t1_dt['signal']==1].value.max()
zero = t1_dt[t1_dt['signal']==0].value.max()
print("short:{0}, long:{1}, zero:{2}, mapping:{3}".format(short, long, zero, roto1._best_mapping))


short:0.04752687979410584, long:-0.1020546979815696, zero:0.4159098338216611, mapping:{0: -1, 1: 1, 2: 0}


In [10]:
roto1 = fetch_rotor('1078761302')
signal_data = roto1.predict(positions_data)
signal_data.name = 'signal'
t1_dt = pd.concat([positions_data, signal_data], axis=1)
short = t1_dt[t1_dt['signal']==-1].value.max()
long = t1_dt[t1_dt['signal']==1].value.max()
zero = t1_dt[t1_dt['signal']==0].value.max()
print("short:{0}, long:{1}, zero:{2}, mapping:{3}".format(short, long, zero, roto1._best_mapping))


short:-0.1020546979815696, long:0.4159098338216611, zero:0.04752687979410584, mapping:{0: 0, 1: -1, 2: 1}


In [12]:
#mapping:{0: -1, 1: 1, 2: 0}

import joblib

infos = joblib.load('/workspace/worker/pj/Chrono/lumina/abily/records/aicso2/ims/metrics/3_0_-1_1.pkl')

In [13]:
infos.empyrical

EmpyricalTuple(name='3_0_-1_1', annual_return=3.0663966509891374, annual_volatility=0.25638374889830917, calmar=29.968274123590998, sharpe=5.610961810163442, max_drawdown=-0.10232142960062131, sortino=14.26811816409296, returns_series=trade_time
2022-07-25    0.0054
2022-07-26    0.0167
2022-07-27    0.0218
2022-07-28   -0.0032
2022-07-29   -0.0032
               ...  
2024-11-29    0.0223
2024-12-02    0.0086
2024-12-03   -0.0004
2024-12-04   -0.0090
2024-12-05   -0.0028
Name: ret, Length: 575, dtype: float64)